In [16]:
import os
import cv2
import math
from ultralytics import YOLO
from collections import deque
from werkzeug.utils import secure_filename
from flask import Flask, render_template, Response, request, jsonify

In [17]:
CONF_THRESHOLD = 0.6
classNames = ["bouledogue","whitetip", "blacktip"]
#model = YOLO("runs/detect/train16/weights/best.engine")
model = YOLO("runs/detect/train16/weights/best.engine")
#model = YOLO('yolov8m.engine', task='detect')

In [18]:
def make_pred(classNames, img, results):
    detection = False
    labels = []
    boxes_to_draw = []

    for result in results:
        boxes = result.boxes
        for box in boxes:
            conf = math.ceil((box.conf[0] * 100)) / 100
            if conf > CONF_THRESHOLD:
                detection = True
                x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
                cls = int(box.cls[0])
                class_name = classNames[cls]
                label = f'{class_name} : {conf*100:.0f}%'
                labels.append((x1, y1, label))
                boxes_to_draw.append((x1, y1, x2, y2))

    # Draw boxes and labels
    for (x1, y1, x2, y2) in boxes_to_draw:
        cv2.rectangle(img, (x1, y1), (x2, y2), (255, 255, 255), 3)

    for (x1, y1, label) in labels:
        t_size = cv2.getTextSize(label, 0, fontScale=1, thickness=2)[0]
        c2 = x1 + t_size[0], y1 - t_size[1] - 3
        cv2.rectangle(img, (x1, y1), c2, [255, 255, 255], -1, cv2.LINE_AA)
        cv2.putText(img, label, (x1, y1 - 2), 0, 1, [122, 0, 0], thickness=1, lineType=cv2.LINE_AA)

    return detection

In [19]:
def image_detect(image_path):
    img   = cv2.imread(image_path)

    if img is None:
        return None
    
    results = model(img, verbose=False)
    make_pred(classNames, img, results)
                
    result_path = image_path.replace('uploads', 'results')
    cv2.imwrite(result_path, img)
    
    return result_path

In [20]:
def video_detection(path):
    cap = cv2.VideoCapture(path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    tolerance_threshold = int(fps * 2)
    pre_detection_buffer_length = int(fps * 1)
    
    no_detection_count = 0
    in_detection_sequence = False
    video_writer = None
    
    detection_sequence_counter = 0  # Compteur pour les séquences de détection
    pre_detection_buffer = deque(maxlen=pre_detection_buffer_length)  # Buffer pour les images avant la détection
    
    frame_width  = int(cap.get(3))
    frame_height = int(cap.get(4))
    
    # Create a separate video writer for the complete video
    result_path = path.replace('uploads', 'results')
    complete_video_writer = cv2.VideoWriter(f'{result_path}', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))
    
    while True:
        success, img = cap.read()
        if not success:
            break
        
        pre_detection_buffer.append(img.copy())
        results = model(img, stream=True, verbose=False)
        detection = make_pred(classNames, img, results) 
        
        if detection:
            if not in_detection_sequence:
                in_detection_sequence = True
                detection_sequence_counter += 1
                video_writer = cv2.VideoWriter(f'static/results/output_{detection_sequence_counter}.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))
                
                for buffered_img in pre_detection_buffer:
                    video_writer.write(buffered_img)
                pre_detection_buffer.clear()
                
            no_detection_count = 0
            video_writer.write(img)
            
        else:
            no_detection_count += 1
            if in_detection_sequence and no_detection_count <= tolerance_threshold:
                video_writer.write(img)
            
            elif in_detection_sequence and no_detection_count > tolerance_threshold:
                video_writer.release()
                in_detection_sequence = False
                pre_detection_buffer.clear()
                
        complete_video_writer.write(img) 
        
    if in_detection_sequence:
        video_writer.release()
    
    complete_video_writer.release()  # Release the complete video writer
    
    return result_path

In [21]:
        #if recording == True:
        #    ref, buffer = cv2.imencode('.jpg', img)
        #    frame = buffer.tobytes()
        #    yield (b'--frame\r\n' b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

In [22]:
app = Flask(__name__) 

app.config['UPLOAD_FOLDER'] = 'static/uploads/'
app.config['RESULT_FOLDER'] = 'static/results/'
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg', 'gif', 'mp4', 'avi', 'mov', 'PNG', 'JPG', 'JPEG', 'GIF', 'MP4', 'AVI', 'MOV'}

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/', methods=['POST'])
def upload_file():
    file = request.files['file']
    if file.filename == '':
        return jsonify(error="No selected file"), 400
    
    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(filepath)
        
        file_extension = filename.rsplit('.', 1)[1].lower()
        if file_extension in ['mp4', 'avi', 'mov', 'MP4', 'AVI', 'MOV']:
            result_path = video_detection(filepath)
        elif file_extension in ['png', 'jpg', 'jpeg', 'JPG']:
            result_path = image_detect(filepath)
            
        if result_path:
            result_filename = os.path.basename(result_path)
            return jsonify({'filename': result_filename}), 200
    return jsonify({'error': 'Image detection failed'})
    
@app.route('/webcam')
def webcam():
    return Response(video_detection(path=0), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/gallery')
def gallery():
    images = os.listdir('static/results')
    images = [image for image in images if image.endswith(('jpg', 'jpeg', 'png', 'gif', 'JPG'))]
    return render_template('gallery.html', images=images)

if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [24/May/2024 22:23:25] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2024 22:23:25] "GET /static/style.css HTTP/1.1" 304 -


Loading runs\detect\train16\weights\best.engine for TensorRT inference...


127.0.0.1 - - [24/May/2024 22:23:58] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2024 22:23:58] "GET /static/results/20240423_002235A.mp4 HTTP/1.1" 200 -
